# テキストを文字ごとに再帰的に分割する方法

この [text splitter](/docs/concepts/text_splitters/) は、一般的なテキスト向けに推奨されるものです。  
リスト化された文字を基準にパラメータ化されており、指定した順番でテキストを分割していきます。  
チャンクが十分に小さくなるまで順に分割を試みます。デフォルトのリストは  `["\n\n", "\n", " ", ""]`です。  
これにより、段落（次に文、次に単語）を可能な限り一緒に保持し、意味的に最も関連性の高いテキスト単位を保つ効果があります。

主な特徴
1. テキストの分割方法: 文字リストに基づいて分割します。
2. チャンクサイズの測定方法: 文字数に基づいて測定します。

以下は、使用例を示したものです。
- 文字列コンテンツを直接取得したい場合: `.split_text` を使用します。
- LangChainの[Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) オブジェクトを作成したい場合（例: 後続のタスクで使用するため）: `.create_documents` を使用します。

In [ ]:
%pip install -qU langchain-text-splitters

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# サンプルドキュメントを読み込む
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # 小さなチャンクサイズを設定（例示のため）
    chunk_size=100,  # チャンクの最大サイズ
    chunk_overlap=20,  # チャンク間の重複部分のサイズ
    length_function=len,  # チャンクサイズを測定する関数
    is_separator_regex=False,  # セパレーターリストを正規表現として解釈するかどうか
)

# ドキュメントを分割してLangChainのDocumentオブジェクトを作成
texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])  # 最初のチャンクを表示
print(texts[1])  # 2番目のチャンクを表示

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and'
page_content='of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.'


In [2]:
# テキストを文字列リストとして分割
text_splitter.split_text(state_of_the_union)[:2]

['Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and',
 'of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.']

`RecursiveCharacterTextSplitter`のパラメータ解説

- `chunk_size`:  
チャンクの最大サイズ。サイズはlength_functionで決定されます。上記では最大100文字。
- `chunk_overlap`:  
チャンク間の重複部分の目標サイズ。チャンクが分割される際にコンテキストが失われるのを防ぐために使用します。上記では20文字。
- `length_function`:  
チャンクサイズを決定する関数。上記では文字数を使用して測定（len）。
- `is_separator_regex`:  
セパレーターリスト（デフォルトでは ["\n\n", "\n", " ", ""]）を正規表現として解釈するかどうか。上記ではFalseに設定されています。

## 単語境界がない言語でのテキスト分割

一部の言語（例: 中国語、日本語、タイ語）では、[単語境界](https://en.wikipedia.org/wiki/Category:Writing_systems_without_word_boundaries)が存在しません。  
この場合、デフォルトのセパレーターリスト` ["\n\n", "\n", " ", ""] `を使用すると、単語がチャンク間で分割される可能性があります。  
単語を分割せずに保持するためには、セパレーターリストをカスタマイズし、追加の句読点を含めることができます。

カスタマイズするセパレーター
1. ASCIIピリオド（.）、[Unicode全角ピリオド](https://en.wikipedia.org/wiki/Halfwidth_and_Fullwidth_Forms_(Unicode_block))（．）、および句点（。）
    - 中国語や日本語で使用される終止符。
2. ゼロ幅スペース（U+200B）
    - タイ語、ミャンマー語、クメール語、日本語などで使用される。
3. ASCIIコンマ（,）、Unicode全角コンマ（，）、および句読点（、）
    - 中国語や日本語で使用されるコンマ。

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",     # 段落区切り
        "\n",       # 改行
        " ",        # スペース
        ".",        # ピリオド
        ",",        # コンマ
        "\u200b",   # ゼロ幅スペース
        "\uff0c",   # 全角コンマ
        "\u3001",   # 句読点
        "\uff0e",   # 全角ピリオド
        "\u3002",   # 句点
        "",         # 最後の分割手段として空文字列
    ],
    # その他の引数（例: chunk_size, chunk_overlapなど）
)